In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
df = pd.read_csv('Churn.csv')

In [3]:
# Drop Customer ID (not useful for prediction) if it exists
if 'Customer ID' in df.columns:
    df = df.drop('Customer ID', axis=1)

# Convert 'Total Charges' to numeric (some may be blank or spaces)
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')

# Fill missing numeric values with median
df['Total Charges'] = df['Total Charges'].fillna(df['Total Charges'].median())

# Encode categorical columns
for col in df.select_dtypes(include='object').columns:
    if col != 'Churn':
        df[col] = LabelEncoder().fit_transform(df[col])

# Encode target
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

# Features and target
X = df.drop('Churn', axis=1)
y = df['Churn']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/Users/mac/Projects/UniversityProjects/Isaac_Mwesigwa/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6647 - loss: 0.5860 - val_accuracy: 0.7604 - val_loss: 0.4610
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.7852 - loss: 0.4438 - val_accuracy: 0.7746 - val_loss: 0.4506
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.7950 - loss: 0.4229 - val_accuracy: 0.7791 - val_loss: 0.4443
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - accuracy: 0.7971 - loss: 0.4158 - val_accuracy: 0.7835 - val_loss: 0.4422
Epoch 5/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.7925 - loss: 0.4163 - val_accuracy: 0.7888 - val_loss: 0.4421
Epoch 6/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.7939 - loss: 0.4232 - val_accuracy: 0.7933 - val_loss: 0.4423
Epoch 7/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.8054 - loss: 0.4124 - val_accuracy: 0.7933 - val_loss: 0.4420
Epoch 8/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.8080 - loss: 0.4117 - va

In [6]:
# Save the model
model.save('churn_tf_model.h5')

# To reload the model later:
reloaded_model = tf.keras.models.load_model('churn_tf_model.h5')

# Predict on test set
y_pred = (reloaded_model.predict(X_test) > 0.5).astype(int)

# Evaluate
loss, acc = reloaded_model.evaluate(X_test, y_test)
print(f"Reloaded model accuracy: {acc:.2f}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.7861 - loss: 0.4220
Reloaded model accuracy: 0.78


In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
print(f"Model Accuracy: {accuracy:.3f}")

Model Accuracy: 0.784
